In [1]:
import os
import mne
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from scipy import signal
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [2]:
class SSVEPDataset(Dataset):
    """Enhanced SSVEP dataset loader with frequency feature extraction"""
    
    # SSVEP frequencies for each class
    FREQUENCIES = {
        'Left': 10,
        'Right': 13,
        'Forward': 7,
        'Backward': 8
    }
    
    def __init__(self, csv_path, task='SSVEP', eeg_reference='average', transform=None):
        """
        Args:
            csv_path: Path to metadata CSV
            task: Task type (only SSVEP supported)
            eeg_reference: Reference method for EEG
            transform: Optional transforms to be applied
        """
        self.base_path = '/kaggle/input/mtcaic3'
        self.metadata = pd.read_csv(csv_path)
        self.eeg_reference = eeg_reference
        self.transform = transform
        
        if task:
            self.metadata = self.metadata[self.metadata['task'] == task]
            
        # Create label encoding
        print(sorted(self.FREQUENCIES.keys()))
        self.classes = sorted(self.FREQUENCIES.keys())
        self.label_encoding = {label: i for i, label in enumerate(self.classes)}
        
        # Precompute frequency bands of interest
        self.freq_bands = {
            label: (freq-1, freq+1) for label, freq in self.FREQUENCIES.items()
        }
        
        # Sampling rate (Hz)
        self.sfreq = 250
        
    def apply_reference(self, raw):
        """Apply EEG referencing"""
        try:
            if self.eeg_reference == 'average':
                raw.set_eeg_reference('average', verbose=False)
            elif self.eeg_reference in raw.ch_names:
                raw.set_eeg_reference([self.eeg_reference], verbose=False)
            elif self.eeg_reference is not None:
                raise ValueError(f"Invalid EEG reference: {self.eeg_reference}")
        except Exception as e:
            raise RuntimeError(f"EEG referencing failed: {e}")
        return raw
    
    def extract_frequency_features(self, eeg_data):
        """Extract power in SSVEP frequency bands"""
        n_channels, n_samples = eeg_data.shape
        features = np.zeros((n_channels, len(self.freq_bands)))
        
        for i, (label, (low, high)) in enumerate(self.freq_bands.items()):
            # Compute power spectral density using Welch's method
            freqs, psd = signal.welch(eeg_data, fs=self.sfreq, nperseg=min(256, n_samples))
            
            # Find indices of frequency band
            idx = np.logical_and(freqs >= low, freqs <= high)
            
            # Compute average power in band for each channel
            features[:, i] = np.mean(psd[:, idx], axis=1)
            
        return features
    
    def normalize(self, data):
        """Normalize data channel-wise"""
        mean = np.mean(data, axis=1, keepdims=True)
        std = np.std(data, axis=1, keepdims=True)
        return (data - mean) / (std + 1e-6)
    
    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        id_num = row['id']
        dataset = 'train' if id_num <= 4800 else 'validation' if id_num <= 4900 else 'test'
        
        # Load EEG data
        eeg_path = os.path.join(
            self.base_path, row['task'], dataset,
            row['subject_id'], str(row['trial_session']), "EEGdata.csv"
        )
        
        df = pd.read_csv(eeg_path)
        df["Time"] -= df["Time"].iloc[0]
        
        # Extract trial data
        trial_num = int(row['trial'])
        samples_per_trial = 1750
        start_idx = (trial_num - 1) * samples_per_trial
        end_idx = start_idx + samples_per_trial
        df = df.iloc[start_idx:end_idx]
        
        # EEG processing
        eeg_channels = ['PO7', 'OZ', 'PO8']
        if not all(ch in df.columns for ch in eeg_channels):
            raise ValueError(f"Missing required EEG channels in file: {eeg_path}")
        
        eeg_data = df[eeg_channels].values.T
        info = mne.create_info(ch_names=eeg_channels, sfreq=self.sfreq, ch_types='eeg')
        raw = mne.io.RawArray(eeg_data, info, verbose=False)
        
        # Apply notch filter at 50Hz and bandpass filter in SSVEP range
        raw.notch_filter(freqs=50, verbose=False)
        raw.filter(l_freq=6, h_freq=30, verbose=False)  # Wider band to capture all frequencies
        
        # Apply referencing
        raw = self.apply_reference(raw)
        
        # Get filtered data
        eeg_filtered = raw.get_data()
        
        # Extract frequency features
        freq_features = self.extract_frequency_features(eeg_filtered)
        
        # Normalize
        eeg_normalized = self.normalize(eeg_filtered).astype(np.float32)
        freq_features = self.normalize(freq_features).astype(np.float32)
        
        # Motion data
        motion_channels = ['AccX', 'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3']
        motion_data = df[motion_channels].values.T
        motion_normalized = self.normalize(motion_data).astype(np.float32)
        
        # Convert to tensors
        eeg_tensor = torch.from_numpy(eeg_normalized)
        freq_tensor = torch.from_numpy(freq_features)
        motion_tensor = torch.from_numpy(motion_normalized)
        label_tensor = torch.tensor(self.label_encoding[row['label']], dtype=torch.long)
        
        if self.transform:
            eeg_tensor = self.transform(eeg_tensor)
            
        return eeg_tensor, freq_tensor, motion_tensor, label_tensor

In [3]:
class EnhancedSSVEPModel(nn.Module):
    """Enhanced model for SSVEP classification with frequency attention"""
    
    def __init__(self, num_classes=4, eeg_channels=3, motion_channels=6, freq_bands=4):
        super(EnhancedSSVEPModel, self).__init__()
        
        # EEG branch - deeper architecture with residual connections
        self.eeg_branch = nn.Sequential(
            # First block
            nn.Conv1d(eeg_channels, 64, kernel_size=5, padding=2),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.MaxPool1d(2),  # 875
            
            # Second block with residual
            ResidualBlock(64, 128, kernel_size=5, stride=2, downsample=True),  # 438
            
            # Third block
            ResidualBlock(128, 256, kernel_size=5, stride=2, downsample=True),  # 219
            
            # Fourth block
            ResidualBlock(256, 512, kernel_size=5, stride=2, downsample=True),  # 110
            
            # Attention pooling
            AttentionPooling(512),
            
            # Final projection
            nn.Linear(512, 256)
        )
        
        # Rest of the model remains the same...
        self.freq_branch = nn.Sequential(
            nn.Linear(eeg_channels * freq_bands, 128),
            nn.ELU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64)
        )
        
        self.motion_branch = nn.Sequential(
            nn.Conv1d(motion_channels, 32, kernel_size=5, padding=2),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.MaxPool1d(2),  # 875
            
            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.MaxPool1d(2),  # 438
            
            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.BatchNorm1d(128),
            nn.ELU(),
            nn.AdaptiveAvgPool1d(1)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(256 + 64 + 128, 512),
            nn.ELU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ELU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, eeg, freq, motion):
        # EEG branch
        eeg_features = self.eeg_branch(eeg)  # [B, 256]
        
        # Frequency branch
        batch_size = freq.size(0)
        freq_features = self.freq_branch(freq.view(batch_size, -1))  # [B, 64]
        
        # Motion branch
        motion_features = self.motion_branch(motion).squeeze(-1)  # [B, 128]
        
        # Combine features
        combined = torch.cat([eeg_features, freq_features, motion_features], dim=1)
        logits = self.classifier(combined)
        
        return logits


class ResidualBlock(nn.Module):
    """Residual block with downsampling"""
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1, downsample=False):
        super(ResidualBlock, self).__init__()
        self.downsample = downsample
        
        padding = kernel_size // 2
        
        self.conv1 = nn.Conv1d(in_channels, out_channels, 
                             kernel_size=kernel_size, 
                             padding=padding, 
                             stride=stride)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.elu = nn.ELU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, 
                             kernel_size=kernel_size, 
                             padding=padding)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        if downsample or in_channels != out_channels:
            self.downsample_conv = nn.Conv1d(in_channels, out_channels, 
                                           kernel_size=1, 
                                           stride=stride)
            self.downsample_bn = nn.BatchNorm1d(out_channels)
        else:
            self.downsample_conv = None
        
    def forward(self, x):
        residual = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.elu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample_conv is not None:
            residual = self.downsample_conv(residual)
            residual = self.downsample_bn(residual)
            
        out += residual
        out = self.elu(out)
        
        return out


class AttentionPooling(nn.Module):
    """Attention-based temporal pooling"""
    def __init__(self, channels):
        super(AttentionPooling, self).__init__()
        self.attention = nn.Sequential(
            nn.Conv1d(channels, channels//8, kernel_size=1),
            nn.ELU(),
            nn.Conv1d(channels//8, 1, kernel_size=1),
            nn.Softmax(dim=2)
        )
        
    def forward(self, x):
        # x shape: [B, C, T]
        weights = self.attention(x)  # [B, 1, T]
        out = torch.sum(x * weights, dim=2)  # [B, C]
        return out

In [4]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=50, device='cuda'):
    """Enhanced training loop with comprehensive metrics tracking"""
    
    history = {
        'train_loss': [],
        'train_f1': [],
        'val_loss': [],
        'val_f1': [],
        'lr': []
    }

    best_loss = float('inf')
    best_f1   = 0
    best_model_path = None
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        # Training phase with progress bar
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for eeg, freq, motion, labels in pbar:
            eeg = eeg.to(device)
            freq = freq.to(device)
            motion = motion.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(eeg, freq, motion)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            # Update progress bar
            pbar.set_postfix({'loss': loss.item()})
        
        # Calculate training metrics
        train_loss = running_loss / len(train_loader)
        train_f1 = f1_score(all_labels, all_preds, average='macro')
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for eeg, freq, motion, labels in val_loader:
                eeg = eeg.to(device)
                freq = freq.to(device)
                motion = motion.to(device)
                labels = labels.to(device)
                
                outputs = model(eeg, freq, motion)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_loss /= len(val_loader)
        val_f1 = f1_score(val_labels, val_preds, average='macro')
        
        # Update learning rate
        current_lr = optimizer.param_groups[0]['lr']
        scheduler.step()
        new_lr = optimizer.param_groups[0]['lr']
        
        # Store history
        history['train_loss'].append(train_loss)
        history['train_f1'].append(train_f1)
        history['val_loss'].append(val_loss)
        history['val_f1'].append(val_f1)
        history['lr'].append(current_lr)
        
        # Print epoch summary
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.4f} | Train F1: {train_f1:.4f}")
        print(f"  Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f}")
        print(f"  LR: {current_lr:.6f}")
        
        # Save best model
        if (val_f1 > best_f1) or (val_loss < best_loss):
            if (val_f1 > best_f1):
                best_f1   = val_f1
            else:
                best_loss = val_loss
                
            best_model_path = f'ssvep_model_epoch{epoch+1}_f1{val_f1:.4f}_loss{val_loss:.4f}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_f1': val_f1,
                'val_loss': val_loss,
            }, best_model_path)
            print(f"  New best model saved: F1={best_f1:.4f} - Loss{best_loss:.4f}")
        
        print("-" * 60)
    
    return history, best_model_path

def plot_training_history(history):
    """Plot training and validation metrics"""
    plt.figure(figsize=(12, 4))
    
    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # F1 score plot
    plt.subplot(1, 2, 2)
    plt.plot(history['train_f1'], label='Train F1')
    plt.plot(history['val_f1'], label='Val F1')
    plt.title('Training and Validation F1 Score')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [5]:
# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# Initialize datasets
train_dataset = SSVEPDataset("/kaggle/input/mtcaic3/train.csv", task='SSVEP')
val_dataset = SSVEPDataset("/kaggle/input/mtcaic3/validation.csv", task='SSVEP')

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Using cuda device


In [6]:
for data in train_loader:
    print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)
    break

torch.Size([32, 3, 1750]) torch.Size([32, 3, 4]) torch.Size([32, 6, 1750]) torch.Size([32])


In [7]:
# Initialize model
model = EnhancedSSVEPModel(num_classes=4).to(device)

criterion = nn.CrossEntropyLoss()

# Optimizer with weight decay
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

In [8]:
# Print model summary
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

Trainable parameters: 3,350,949


In [ ]:
# Train the model
history, best_model_path = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=50)

Epoch 1/50 [Train]: 100%|██████████| 75/75 [03:17<00:00,  2.64s/it, loss=1.2] 


Epoch 1/50
  Train Loss: 1.3328 | Train F1: 0.3541
  Val Loss: 1.1829 | Val F1: 0.4131
  LR: 0.001000
  New best model saved: F1=0.4131 - Lossinf
------------------------------------------------------------


Epoch 2/50 [Train]: 100%|██████████| 75/75 [03:09<00:00,  2.53s/it, loss=1.28] 


Epoch 2/50
  Train Loss: 1.2060 | Train F1: 0.4450
  Val Loss: 1.1339 | Val F1: 0.4146
  LR: 0.000999
  New best model saved: F1=0.4146 - Lossinf
------------------------------------------------------------


Epoch 3/50 [Train]: 100%|██████████| 75/75 [03:14<00:00,  2.59s/it, loss=1.3]  


...

Epoch 45/50 [Train]: 100%|██████████| 75/75 [03:18<00:00,  2.65s/it, loss=0.61] 


Epoch 45/50
  Train Loss: 0.6234 | Train F1: 0.7554
  Val Loss: 1.6765 | Val F1: 0.4821
  LR: 0.000035
------------------------------------------------------------


Epoch 46/50 [Train]: 100%|██████████| 75/75 [03:18<00:00,  2.64s/it, loss=0.543]


Epoch 46/50
  Train Loss: 0.6112 | Train F1: 0.7553
  Val Loss: 1.6251 | Val F1: 0.5231
  LR: 0.000024
------------------------------------------------------------


Epoch 47/50 [Train]:  75%|███████▍  | 56/75 [02:29<00:50,  2.67s/it, loss=0.44] 

In [ ]:
# Plot training history
plot_training_history(history)